In [7]:
print("Settings.py should be e.g. flows_1")
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2 
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

LSTM_IO_SIZE = 18*13
LSTM_HIDDEN_SIZE = 18*13

RGB_CHANNELS = 3
TIMESTEPS = 30 # size videos
BATCH_SIZE = 1 #until now just batch_size = 1
SLICE_FROM_TIMESTEP = 1 #slices from timestep SLICE_FROM_TIMESTEP to the last one

NR_EPOCHS = 20

VALIDATION_SPLIT = 0.0 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

USE_EXISTING_NORMALIZATION = True

learning_rate = 0.01 # reduce factos of 10 .. some epoch later.
momentum = 0.9
print("Hyperparameters defined")

Settings.py should be e.g. flows_1
Hyperparameters defined


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import os.path as path
import copy
# from torch.utils.data.sampler import SequentialSampler


from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer

import matplotlib.pyplot as plt

print("done")

done


In [9]:
#change configuration in settings.py
crop_im_size = [186,250]
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
c.config['transformers']['DimensionResize']['dimension'] = TIMESTEPS
#c.config['indexing']['people_selection'] = [3]
#c.config['indexing']['sequences_selection'] = ['nm-01']
c.config['pose']['load'] = False
c.config['flow']['load'] = True
c.config['heatmaps']['load'] = False
#c.config['annotations']['preprocess']=True
#c.config['scenes']['sequences'] = ['nm']
#c.config['scenes']['angles'] = ['108']
c.config['dataset_output'] = {
#         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'data': ['flows'],
        'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)

In [11]:
print(len(dataset))
a,b = dataset[1]

print(len(a["flows"]))

135
30


In [5]:
print(a.keys())

dict_keys(['flows'])


# Settings 

## Dataset

In [13]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)


mean = [126.1972, 127.50477, 1.3415707]
std_dev = [6.436958,  1.7359118, 5.022056 ]

#mean = [126.04878374565972, 127.50835754123264, 1.5171344184027777]
#std_dev = [7.03813454, 2.04901181, 5.66295848]
if not(USE_EXISTING_NORMALIZATION):
    print('Normalizing')
    image_list_ch_0 = []
    image_list_ch_1 = []
    image_list_ch_2 = []
    for data in dataset:
        print('here')
        for image in data[0]['flows']:
            image_list_ch_0.append(image[0].numpy())
            image_list_ch_1.append(image[1].numpy())
            image_list_ch_2.append(image[2].numpy())
    print("List of channels created")
    mean = [np.mean(image_list_ch_0),np.mean(image_list_ch_1),np.mean(image_list_ch_2)]
    std_dev = np.sqrt([np.var(image_list_ch_0),np.var(image_list_ch_1),np.var(image_list_ch_2)])
    
    

    
# print("Data in original", len(data_in))
# data_in_norm = data_in
# for i,data in enumerate(data_in):
#     for image in data:
#         ch_0 = (image[0][0] - mean[0]) / std_dev[0]
#         ch_1 = (image[0][1] - mean[1]) / std_dev[1]
#         ch_2 = (image[0][2] - mean[2]) / std_dev[2]
#         print(ch_3)
# #         channel_list = [ch_0, ch_1, ch_2]
# #         data_in_norm.append(channel_list)
  
# print("New",data_in_norm)
# print("Original",data_in)
# dataset_new = dataset
# print("Normalizing images... DOES NOT WORK LIKE THIS")
# for i, data in enumerate(dataset):
#     print("Dataset item nr ", i)
#     print("Iterating through", len(data[0]['flows']), "images")
#     for j,image in enumerate(data[0]['flows']):
#         image[0] = (image[0] - mean[0]) / std_dev[0]
#         image[1] = (image[1] - mean[1]) / std_dev[1]
#         image[2] = (image[2] - mean[2]) / std_dev[2]
#         print("Before", dataset_new[i][0]['flows'][j])
#         dataset_new[i][0]['flows'][j] = image
#         print("Image",image)
#         print("From dataset",dataset_new[i][0]['flows'][j])
    
#         print("Dataset image",dataset_new[i][0]['flows'][0])
# print("Images normalized with mean and std dev",mean, std_dev)
# print("Display normalized image as array:")
# print(image[1])

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

Dataset size: 135
Indices size: 135
Split: 0


## neural network

In [14]:
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.avialable_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.conv1 = nn.Conv2d(3,6,3) #input 640x480
        self.pool1 = nn.MaxPool2d(2,2) #input 638x478 output 319x239
        self.conv2 = nn.Conv2d(6,16,3) # input 319x239 output 317x237
        self.pool2 = nn.MaxPool2d(2,2) # input 317x237 output 158x118
        self.conv3 = nn.Conv2d(16,6,3) # input 158x118 output 156x116
        self.pool3 = nn.MaxPool2d(2,2) # input 156x116 output 78x58
        self.conv4 = nn.Conv2d(6,3,3)  # input 78x58 output 76x56
        self.pool4 = nn.MaxPool2d(2,2) # input 76x56 output 39x29
        self.conv5 = nn.Conv2d(3,1,3)  # input 39x29 output 37x27
        self.pool5 = nn.MaxPool2d(2,2) #output 37x27 output 18x13
        self.lstm1 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# before: TIMESTEPS. But TIMESTEPS should be wrong
#         self.lstm2 = nn.LSTM(LSTM_IO_SIZE,
#                             LSTM_HIDDEN_SIZE,
#                             TIMESTEPS)# before: TIMESTEPS. But TIMESTEPS should be wrong
        self.fc1 = nn.Linear(LSTM_IO_SIZE,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
        #initialize hidden states of LSTM
        self.hidden = self.init_hidden()

        #print("Hidden:", _hidden)
    def init_hidden(self):
        return (torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device),#before: TIMESTEPS 
                torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device))#before: TIMESTEPS
    def forward(self,x):
#         print("Input list len:",len(x))
#         print("Input elemens size:", x[0].size())
#         batch_size = x[0].size()[0]

        x_arr = torch.zeros(TIMESTEPS,BATCH_SIZE,1,IMAGE_AFTER_CONV_SIZE_H,IMAGE_AFTER_CONV_SIZE_W).to(self.avialable_device)
        #print("X arr size", x_arr.size())
        for i in range(TIMESTEPS):#parallel convolutions which are later concatenated for LSTM
            x_tmp_c1 = self.pool1(F.relu(self.conv1(x[i])))
            x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
            x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
            x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
            x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
            x_arr[i] = x_tmp_c5
        
        x = torch.cat((x_arr[0],
              x_arr[1],
              x_arr[2],
              x_arr[3],
              x_arr[4],
              x_arr[5],
              x_arr[6],
              x_arr[7],
              x_arr[8],
              x_arr[9]),0)
        x, hidden = self.lstm1(x_arr.view(TIMESTEPS,BATCH_SIZE,-1), self.hidden)#before: TIMESTEPS
#         x, hidden = self.lstm2(x, self.hidden)
        # the reshaping was taken from the documentation... and makes scense
        x = x.view(TIMESTEPS,BATCH_SIZE,LSTM_HIDDEN_SIZE) #output.view(seq_len, batch, num_dir*hidden_size)
#         x = torch.squeeze(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        #x = x.squeeze(1)
        x = x.permute(1,2,0)
        #print ("Size network output", x.shape)
        return x
print("Class defined")

Class defined


In [8]:
### define and execute testing function
def test_all_preds(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            data_in = [s.to(device) for s in inputs['flows']]
            labels = labels.to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(data_in)
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')
print("Definition done")

Definition done


### Test-Training

In [9]:
#TRAINING
test_net = TEST_CNN_LSTM()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
test_net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)


cuda:0


### Test set and evaluate computing time etc.

In [10]:
#PREPARATION FOR TRAINING
loss_array = []
learning_rate_array = []

print('Start training...')
print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
running_loss = 0.0
#print("Data set length:", len((train_loader)), "Validation length:", len(test_loader))
print("Batch size:", BATCH_SIZE)
print("Evaluating first element...")
start_time = time.time()
i, batch = next(iter(enumerate(train_loader)))
inputs, labels = batch
data_in = [s.to(device) for s in inputs['flows']]
# print("Data in original", data_in)
# data_in_norm = data_in
# for i,image in enumerate(data_in):
#     print("Normalizing:",i)
# #     data_in_norm[i][1] = data[i][1]
#     print()
#     data_in_norm[0] = (image[0] - mean[0]) / std_dev[0]
#     data_in_norm[1] = (image[1] - mean[1]) / std_dev[1]
#     data_in_norm[2] = (image[2] - mean[2]) / std_dev[2]
# print(data_in_norm[0])
# print("Normalization done")

print("Proof for normalized data:", data_in[0][0])
labels = labels.to(device)
print("Time steps:{}, input sequence length:{}".format(TIMESTEPS,len(data_in)))
#print("NN input: ",len(flows),len(flows[0]),len(flows[0][0]),len(flows[0][0][0]),len(flows[0][0][0][0]))
optimizer.zero_grad() 
outputs = test_net(data_in)
print("Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]")
print("Output format:", len(outputs), outputs.size())
print("Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)")
print("Labels:", len(labels), labels.size())
print("Slicing loss. Using loss from:",SLICE_FROM_TIMESTEP,"to",TIMESTEPS)
#print("Labels content:", labels)
#original: loss = criterion(outputs.float(),labels.long())
loss = criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long())
loss.backward() 
optimizer.step()

running_loss += loss.data.item()
elapsed_time = time.time() - start_time;
loss_array.append(running_loss)
learning_rate_array.append(learning_rate)
print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
print("Time needed:{}s".format(elapsed_time))
print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)/60))
print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*NR_EPOCHS/3600))

Start training...
Expected loss with 3 different classes and 30 data elements: 32.95836866004329
Batch size: 1
Evaluating first element...
Proof for normalized data: tensor([[[ 0.2772,  0.2772,  0.2772,  ...,  0.2772,  0.2772,  0.2772],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2772,  0.2772,  0.2772],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.1352,  0.2772,  0.2772],
         ...,
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2772,  0.2772,  0.2772],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2772,  0.2772,  0.2772],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2772,  0.2772,  0.2772]],

        [[-0.2481,  0.2399,  0.2399,  ...,  0.2399,  0.2399,  0.2399],
         [-0.2481,  0.2399,  0.2399,  ...,  0.2399,  0.2399,  0.2399],
         [-0.2481,  0.2399, -0.2481,  ...,  0.2399,  0.2399,  0.2399],
         ...,
         [ 0.2399,  0.2399, -0.2481,  ..., -0.2481, -0.2481, -0.2481],
         [-0.2481,  0.2399,  0.2399,  ..., -0.2481, -0.2481, -0.2481],
         [ 0.2399,  0.2

### Start Training here

In [32]:
for epoch in range(NR_EPOCHS): 
    running_loss = 0.0
    start_time = time.time()
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        
        outputs = test_net(data_in)
        loss = criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long())
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.data.item()

    print("Loss epoch {}: {}, took {}s".format(epoch, running_loss,time.time()-start_time))
    loss_array.append(running_loss)
    learning_rate_array.append(learning_rate)

#test_all_preds(test_net) 
print('...Training finished')
# plt.plot(loss_array)

Epoch: 0
Loss epoch 0: 4.815722219645977, took 33.41770076751709s
Epoch: 1
Loss epoch 1: 4.760320007801056, took 33.43797540664673s
Epoch: 2
Loss epoch 2: 4.814400762319565, took 33.32983875274658s
Epoch: 3
Loss epoch 3: 4.826962321996689, took 33.5497944355011s
Epoch: 4
Loss epoch 4: 4.7932266145944595, took 33.561710357666016s
Epoch: 5
Loss epoch 5: 4.8098878636956215, took 33.350581884384155s
Epoch: 6
Loss epoch 6: 4.801881864666939, took 33.707473039627075s
Epoch: 7
Loss epoch 7: 4.806560091674328, took 33.43438959121704s
Epoch: 8
Loss epoch 8: 4.807913966476917, took 33.41324996948242s
Epoch: 9
Loss epoch 9: 4.794204398989677, took 33.36453413963318s
Epoch: 10
Loss epoch 10: 4.828056283295155, took 33.69233965873718s
Epoch: 11
Loss epoch 11: 4.808852426707745, took 33.36387228965759s
Epoch: 12
Loss epoch 12: 4.793481923639774, took 33.50886011123657s
Epoch: 13
Loss epoch 13: 4.825078494846821, took 33.56784152984619s
Epoch: 14
Loss epoch 14: 4.783532977104187, took 33.467684268951

In [30]:
#DYNAMIC CHANGES:
NR_EPOCHS = 600
learning_rate = 0.005
momentum = 0.9
optimizer = optim.SGD(test_net.parameters(), lr=learning_rate, momentum=momentum)
print("LR changed")

LR changed


In [15]:
#EVALUATION
print("Expected loss:{}, last loss:{}".format((len(dataset)-split)*np.log(3),loss_array[-1]))
print("Batch size:", BATCH_SIZE)
print("Sequence length:",TIMESTEPS)
print("Total epochs learnt:", len(loss_array))
# plt.plot(learning_rate_array)
# plt.plot(loss_array)
test_all_preds(test_net)


NameError: name 'loss_array' is not defined

KeyboardInterrupt: 

### Expected loss for untrained set with Cross Entropy:
k = number of classes
N = number of labeled data in dataset
loss_per_prediction = -log(1/k) = log(k)
total_loss = sum(log(k)) = N*log(k)

In [14]:
plt.clf()
plot_path = "/home/matthias/Desktop/"
loss_str = "{0:.3f}".format(loss_array[-1])
information = "flows_L" + loss_str + "_timesteps{}_nrsequences{}_batch{}".format(TIMESTEPS,dataset_size,BATCH_SIZE)
plt.xlabel('Epochs')
plt.ylabel('Loss (Cross Entropy)')
plt.plot(loss_array)
plt.title(information)
plt.savefig(plot_path + information + ".eps")
plt.clf()
plt.xlabel('Epochs')
plt.ylabel('Learning rate')
plt.plot(learning_rate_array)
plt.title(information)
plt.savefig(plot_path + information + "LR.eps")

FileNotFoundError: [Errno 2] No such file or directory: '/home/matthias/Desktop/flows_L7.149_timesteps10_nrsequences30_batch1.eps'

In [ ]:
print(outputs.size())
print(outputs[:,:,5:10])
print(outputs.size()[2])
print("Loss half:", criterion(outputs[:,:,SLICE_FROM_TIMESTEP:TIMESTEPS].float(),labels[:,SLICE_FROM_TIMESTEP:TIMESTEPS].long()))
print("Loss full:", criterion(outputs.float(),labels.long()))


In [13]:
image_list_ch_0 = []
image_list_ch_1 = []
image_list_ch_2 = []
for a,data in enumerate(dataset):
    #print(a)
    for image in data[0]['flows']:        
        image_list_ch_0.append(image[0].numpy())
        image_list_ch_1.append(image[1].numpy())
        image_list_ch_2.append(image[2].numpy())
print("List of channels created")
mean = [np.mean(image_list_ch_0),np.mean(image_list_ch_1),np.mean(image_list_ch_2)]
std_dev = np.sqrt([np.var(image_list_ch_0),np.var(image_list_ch_1),np.var(image_list_ch_2)])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
List of channels created


In [16]:
print(mean)
print(std_dev)

std_dev2 = [np.std(image_list_ch_0),np.std(image_list_ch_1),np.std(image_list_ch_2)]
print(std_dev2)

[126.1972, 127.50477, 1.3415707]
[6.436958  1.7359118 5.022056 ]
[6.436958, 1.7359118, 5.022056]
